In [ ]:
import pandas as pd 

ddd = pd.read_csv('csvs/linkedin_f.csv')

In [ ]:
ddd.iloc[0,0]

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import re
from webscraping import By, find_technologies_in_string, linkedin_get_salary
from time import sleep
from random import randint
data_list = []
for i in range(len(ddd)):
    try:
        SOURCE = 'LinkedIn'
        ID = ddd.iloc[i,0]
        url = ddd.iloc[i,-2]
        # print(url)
        r = requests.get(url)
        s = bs(r.text, 'html')
        digest = s.text
        job_description = s.find_all('div',class_="show-more-less-html__markup")[0].text
        sleep(randint(1,4))
        salary = linkedin_get_salary(digest)
        if len(salary)>1:
            lower = salary[0]
            higher = salary[1]
        elif len(salary) == 1:
            lower = salary[0]
            higher = salary[0]
        else:
            lower = None
            higher = None
        techs = find_technologies_in_string(digest)
        # data
        link = s.find_all("a", attrs={"data-tracking-control-name":"public_jobs_topcard_logo"})[0]
        link = link.get("href")
        data = {'ID':ID, 'source':SOURCE, 'min_yearly_salary':lower,
                    'max_yearly_salary':higher,'company_link':link,'description':job_description}
        data.update(techs)
        data_list.append(data)
        print('appended data')
    except:
        pass
df_new = pd.DataFrame(data_list)
df_new

In [ ]:
digest

In [ ]:
from webscraping import linkedin_page_go_to,linkedin_get_text_values_by_class,linkedin_get_href_by_class
from webscraping import linkedin_get_href_by_class,linkedin_driver, linkedin_get_elements_by_css
from time import sleep
from webscraping import linkedin_job_find_next_page_button, linkedin_click
from webscraping import By, find_technologies_in_string
from random import randint

data_list = []
counter = 0
for row in ddd.iterrows():
    counter+=1
    if counter>10:
        break
    ID = row[1][0]
    SOURCE = row[1][-1]
    # print(ID)
    linkedin_page_go_to(driver, row[1][5])
    sleep(randint(1,2))
    # sleep(1)
    try:
        lower,higher = None , None
        button = driver.find_element(By.CLASS_NAME, 'jobs-description__footer-button')
        linkedin_click(button)
        # print('clicking btton')
        description = driver.find_elements(By.CLASS_NAME,'jobs-box__html-content')[0].text
        print('found elements')
        link = linkedin_get_href_by_class('app-aware-link',driver)
        print('found link')
        salary = driver.find_element(By.CLASS_NAME, 'job-details-jobs-unified-top-card__job-insight')
        spans = salary.find_elements(By.TAG_NAME, 'span')
        sal = spans[0].text
        if sal[0] =='£':
            lower  = int(int((sal.split(' ')[0].split('/')[0][1:]).replace(',', ''))*1.22)
            higher = int(int((sal.split(' ')[2].split('/')[0][1:]).replace(',', ''))*1.22)
        elif sal[0] =='$':
            lower  = int(int((sal.split(' ')[0].split('/')[0][1:]).replace(',', '')))
            higher = int(int((sal.split(' ')[2].split('/')[0][1:]).replace(',', '')))
        else:
            pass
        data = {'ID':ID, 'source':SOURCE, 'min_yearly_salary':lower,
                'max_yearly_salary':higher,'company_link':link,'description':description}
        techs = find_technologies_in_string(description)
        data.update(techs)
        data_list.append(data)
        print('appended data')
    except:
        pass
df_new = pd.DataFrame(data_list)
df_new

In [ ]:
salary = driver.find_element(By.CLASS_NAME, 'job-details-jobs-unified-top-card__job-insight')
spans = salary.find_elements(By.TAG_NAME, 'span')
sal = spans[0].text
if sal[0] =='£':
    print(int(int((sal.split(' ')[0].split('/')[0][1:]).replace(',', ''))*1.22))
    print(int(int((sal.split(' ')[2].split('/')[0][1:]).replace(',', ''))*1.22))
else:
    pass

In [ ]:
import requests
import json
from bs4 import BeautifulSoup

# The URL containing the JSON response
url =  "https://github.com/clarketm/proxy-list/blob/master/proxy-list-raw.txt"

# Send a GET request to fetch the JSON response
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = json.loads(response.text)

    # Extract the proxy addresses from the JSON data
    proxy_data = data.get("payload", {}).get("blob", {}).get("rawLines", [])
    
    # Print the proxy addresses
    for proxy in proxy_data:
        print(proxy)
else:
    print("Failed to fetch the JSON data.")